##### 아래 가이드에 따라서 고객 구매데이터를 생성하고 아래사항을 수행하세요(관측치 10000개)

#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.
거래 건수
- 1 ~ 10000번 일련번호

고객별 속성 데이터
- cusno : 고객번호(1000)
- gender : 0,1 정수 난수 생성
- age : 0 ~ 80사이 정수 난수 생성
- region : 1 ~ 10 사이 정수 난수 생성

거래 데이터
- day : 요일
- date : 2020-01-01 ~ 2021-12-31 사이의 날짜 생성
- product_20 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10 ~ 30
- product_21 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10 ~ 30
- price_20 : 1000 ~ 50000 사이 실수 난수 생성
- price_21 : 1000 ~ 50000 사이 실수 난수 생성
- quantity_20 : 1 ~ 100 사이 정수 난수 생성
- quantity_21 : 1 ~ 100 사이 정수 난수 생성
- time_20 : 01 ~ 24 사이 정수 난수 생성
- time_21 : 01 ~ 24 사이 정수 난수 생성
- amount_20 : price_20 * quantity_21
- amount_21 : price_21 * quantity_21
- sales : 20년 대비 21년구매금액이 증가면 1, 감소면 0

#### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요

#### Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.


In [2]:
import string
import pandas as pd
import numpy as np
from datetime import datetime
import random
from datetime import timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 수정 요망 : Date를 인덱스로 한 상황에서 쉽게 바꾸기.. visualization 강의때 참고

In [3]:
np.random.seed(0)
arr = np.zeros((10000,11))
columns = ['cusno','gender','age','region','date','day','time','product_num','price','quantity','amount']
df = pd.DataFrame(arr,columns = columns)

# cusno: 랜덤하게, 중복 있이 1000 범위에서 뽑음
cusno=np.random.randint(1,1001,size=10000)
df.cusno = cusno

# gender, age, region : cusno와 연관되게, 랜덤하게 뽑음
gender = np.random.randint(2,size=1000)
age = np.random.randint(16,81,size=1000)
region=np.random.randint(1,11,size=1000)
c_gen=[]
c_age=[]
c_reg=[]
for i in range(10000):
    c_gen.append(gender[cusno[i]-1])
    c_age.append(age[cusno[i]-1])
    c_reg.append(region[cusno[i]-1])
df.gender = c_gen
df.age = c_age
df.region = c_reg

# date : 랜덤 날짜 생성
date_li = pd.date_range(start = '2020-01-01', end = '2021-12-31',freq='D')
df.date = np.random.choice(date_li,10000)
df.date

# day : date에 맞는 요일 생성
df['day'] = df.date.dt.day_name()
df.loc[:,['date','day']]

# time : '01' 형태로 랜덤하게 생성
time_ar = []
for _ in range(1,25):
    time_ar.append(f'{_}'.zfill(2))
df.time = np.array(np.random.choice(time_ar) for _ in range(10000))

# product_num : 랜덤으로 제품코드 생성
df.product_num = np.random.randint(10,40,size =10000)

# price : 랜덤으로 뽑은 제품코드에 맞는 가격대 설정
price_li_1 = np.random.uniform(1000,17333,size=10) # 10대 제품코드 가격들 중 10개를 미리 뽑음
price_li_2 = np.random.uniform(17333,33666,size=10) # 20대 제품코드 가격들 중 10개를 미리 뽑음
price_li_3 = np.random.uniform(33666,50001,size=10) # 30대 제품코드 가격들 중 10개를 미리 뽑음
def get_price(a): # 제품코드에 대해 미리 설정한 리스트에서 가격을 뽑아 쓰도록 함. (같은 제품코드는 같은 가격으로 나오게 하기 위함)
    if a < 20: df.loc[df.product_num == a,['price']] = price_li_1[a - 10].round(2)
    elif a < 30: df.loc[df.product_num == a,['price']] = price_li_2[a - 20].round(2)
    else: df.loc[df.product_num == a,['price']] = price_li_3[a - 30].round(2)
df.product_num.apply(get_price)

# quantity : 1,100까지 정수 난수 생성
df.quantity = np.random.randint(1,101,size=10000)

# amount : price * quentity
df.amount = (df.price * df.quantity).round(2)

In [4]:
df.head()

,cusno,gender,age,region,date,day,time,product_num,price,quantity,amount
0,685,1,18,10,2021-03-15,Monday,24,20,19027.73,59,1122636.07
1,560,1,71,10,2020-01-11,Saturday,15,33,43636.92,82,3578227.44
2,630,1,52,3,2020-06-07,Sunday,23,10,8005.61,39,312218.79
3,193,1,37,4,2021-12-16,Thursday,11,19,13374.89,31,414621.59
4,836,0,35,5,2020-10-11,Sunday,18,36,45999.08,56,2575948.48


In [5]:
df1 = df.copy()
df1['year'] = df1.date.dt.year
df1.set_index('date',inplace=True)
df1.sort_index(ascending=True,inplace=True)
df1

,cusno,gender,age,region,day,time,product_num,price,quantity,amount,year
date,,,,,,,,,,,
2020-01-01,375,1,66,1,Wednesday,21,26,25997.79,6,155986.74,2020
2020-01-01,632,0,74,6,Wednesday,12,24,32643.17,56,1828017.52,2020
2020-01-01,991,0,19,4,Wednesday,23,27,19927.21,95,1893084.95,2020
2020-01-01,862,1,71,10,Wednesday,13,29,22624.34,47,1063343.98,2020
2020-01-01,196,0,27,8,Wednesday,20,24,32643.17,19,620220.23,2020
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,104,1,75,5,Friday,15,26,25997.79,11,285975.69,2021
2021-12-31,709,0,46,10,Friday,01,19,13374.89,68,909492.52,2021
2021-12-31,873,0,64,6,Friday,08,23,19364.47,21,406653.87,2021


In [6]:
# 같은 cusno에 대한 고객 정보 일치 여부 확인
df1.loc[df1.cusno == 10,['cusno','gender','age','region']]

,cusno,gender,age,region
date,,,,
2020-01-03,10,1,17,9
2020-03-26,10,1,17,9
2020-04-23,10,1,17,9
2020-08-31,10,1,17,9
2020-11-25,10,1,17,9
2021-01-17,10,1,17,9
2021-03-26,10,1,17,9
2021-07-03,10,1,17,9
2021-08-07,10,1,17,9


In [7]:
# 같은 product_num에 대한 price 일치 여부
df1.loc[df1.product_num == 24,['cusno','product_num','price']]

,cusno,product_num,price
date,,,
2020-01-01,632,24,32643.17
2020-01-01,196,24,32643.17
2020-01-03,124,24,32643.17
2020-01-03,10,24,32643.17
2020-01-06,370,24,32643.17
...,...,...,...
2021-12-17,638,24,32643.17
2021-12-20,874,24,32643.17
2021-12-25,824,24,32643.17


In [8]:
# 고객데이터만 따로 뽑기
df_cus = df1.loc[:,['cusno','gender','age','region']]
df_cus = df_cus.drop_duplicates(['cusno'])
df_cus.set_index('cusno',inplace=True)
df_cus.sort_index(ascending=True,inplace=True)
df_cus

,gender,age,region
cusno,,,
1,1,75,5
2,0,21,3
3,0,63,10
4,0,32,6
5,1,16,8
...,...,...,...
996,0,50,10
997,0,53,3
998,1,33,10


In [9]:
arr = np.zeros((1000,6))
columns = ['cusno','quantity_20','amount_20','quantity_21','amount_21','sales']
df_sum = pd.DataFrame(arr,columns = columns)

# cusno
new_cus = list(range(1,1001))
df_sum.cusno = new_cus

# cusno, sales를 제외한 열들
def get_sum(year,column):         # cusno마다 해당년도의 해당 열의 합을 가져오는 함수를 정의
    li = []
    for i in new_cus:
        li.append(df1.loc[(df1.cusno == i)&(df1.year == year),[column]].sum()[0])
    return li
df_sum.quantity_20 = get_sum(2020,'quantity') ; df_sum.quantity_20 = df_sum.quantity_20.astype('int')  # float길래 int로 바꿔줌
df_sum.quantity_21 = get_sum(2021,'quantity') ; df_sum.quantity_21 = df_sum.quantity_21.astype('int')
df_sum.amount_20 = get_sum(2020,'amount')
df_sum.amount_21 = get_sum(2021,'amount')

# sales
df_sum.sales = np.where(df_sum.amount_20<df_sum.amount_21,1,0)
df_sum.set_index('cusno',inplace=True)    # cusno로 인덱스 선택
df_sum.head()

,quantity_20,amount_20,quantity_21,amount_21,sales
cusno,,,,,
1,158,2136911.10,646,13931454.78,1
2,76,1816359.08,83,903987.86,0
3,347,7132630.84,331,7630828.17,1
4,206,4116368.18,248,6834577.21,1
5,385,10585160.91,42,649086.48,0


In [10]:
# 구매내역으로부터 최종 데이터프레임을 구성
# 기준은 구매건수가 아닌 고객들
df_cus_sum = pd.concat([df_cus,df_sum],axis=1)
df_cus_sum

,gender,age,region,quantity_20,amount_20,quantity_21,amount_21,sales
cusno,,,,,,,,
1,1,75,5,158,2136911.10,646,13931454.78,1
2,0,21,3,76,1816359.08,83,903987.86,0
3,0,63,10,347,7132630.84,331,7630828.17,1
4,0,32,6,206,4116368.18,248,6834577.21,1
5,1,16,8,385,10585160.91,42,649086.48,0
...,...,...,...,...,...,...,...,...
996,0,50,10,193,1513697.75,298,8320794.79,1
997,0,53,3,302,6759641.44,234,6613195.77,0
998,1,33,10,64,2269488.64,422,11667295.65,1


In [11]:
df_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   quantity_20  1000 non-null   int32  
 1   amount_20    1000 non-null   float64
 2   quantity_21  1000 non-null   int32  
 3   amount_21    1000 non-null   float64
 4   sales        1000 non-null   int32  
dtypes: float64(2), int32(3)
memory usage: 35.2 KB


In [13]:
df1.loc[(df1.year == 2020)&(df1.product_num >= 30),['quantity']].mean()

quantity    51.10463
dtype: float64

In [14]:
df1.loc[(df1.year == 2021)&(df1.product_num >= 30),['quantity']].mean()

quantity    50.126165
dtype: float64

In [15]:
idx = df_cus_sum.loc[df_cus_sum.sales == 1].index
list(idx)
df1.iloc[list(idx-1),6].unique()

array([26, 27, 29, 19, 18, 23, 37, 21, 15, 17, 20, 11, 38, 25, 30, 13, 36,
       34, 35, 16, 31, 14, 12, 24, 39, 33, 22, 32, 10, 28])

### 시나리오
- 모든 품목에 대해, 20년보다 21년의 평균 물가가 3%높다
- 코로나로 인한 총 매출의 저하(구매 30% 감소)를 막기 위해 가입되어 있는 회원들에게 21년 1월, 2월에 모든 구매건에 5% 할인 적용(20대의 구매 감소율이 비교적 적다)
- 3월 한달 간 20대를 겨냥한 10% 할인 진행 (20대의 quantity 20%증가)
- 21년 7월,8월 동안 특정 품목에 대한 10%할인 진행 (quantity 20%증가)
- 21년 9월 여성을 겨냥한 20% 할인 진행 (quantity 30% 증가)
### 결과로 나올 인사이트? (1년간 진행한 할인행사의 효과를 확인하고, 총 매출의 증가를 확인)

In [16]:
df_test = df.copy()
df_test['year'] = df_test.date.dt.year
df_test['month'] = df_test.date.dt.month
df_test

,cusno,gender,age,region,date,day,time,product_num,price,quantity,amount,year,month
0,685,1,18,10,2021-03-15,Monday,24,20,19027.73,59,1122636.07,2021,3
1,560,1,71,10,2020-01-11,Saturday,15,33,43636.92,82,3578227.44,2020,1
2,630,1,52,3,2020-06-07,Sunday,23,10,8005.61,39,312218.79,2020,6
3,193,1,37,4,2021-12-16,Thursday,11,19,13374.89,31,414621.59,2021,12
4,836,0,35,5,2020-10-11,Sunday,18,36,45999.08,56,2575948.48,2020,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,989,1,61,10,2020-02-13,Thursday,11,22,27545.79,35,964102.65,2020,2
9996,81,1,32,2,2020-04-12,Sunday,17,29,22624.34,15,339365.10,2020,4
9997,276,0,48,1,2021-09-08,Wednesday,04,16,4607.76,46,211956.96,2021,9
9998,677,1,16,10,2020-12-26,Saturday,22,31,42977.40,46,1976960.40,2020,12


In [17]:
df_test = df.copy()
df_test['year'] = df_test.date.dt.year
df_test['month'] = df_test.date.dt.month

con_2020 = (df_test.year == 2020)
con_2021 = (df_test.year == 2021)
con_age_20 = ((df_test.age >= 20)&(df_test.age < 30))
con_age_other = ((df_test.age < 20)|(df_test.age >= 30))

def reamount():
    df_test.amount = df_test.price * df_test.quantity
# 물가 상승
df_test.loc[con_2021,['price']] = df_test.loc[con_2021,['price']] * 1.03
df_test.loc[con_2021,['quantity']] = df_test.loc[con_2021,['quantity']] * 0.75
reamount()
# 여성 구매량 > 남성 구매량
df_test.loc[df_test.gender==0,['quantity']] = df_test.loc[df_test.gender==0,['quantity']] * 1.2
df_test.loc[df_test.gender==1,['quantity']] = df_test.loc[df_test.gender==1,['quantity']] * 0.6
reamount()
# 여성이 주로 사는 품목 설정 15,21,22,23,30,31,32
con_femalepro = ((df_test.product_num == 15)&(df_test.product_num == 21)&(df_test.product_num == 22)&(df_test.product_num == 23)&(df_test.product_num == 30)&(df_test.product_num == 31)&(df_test.product_num == 32))
df_test.loc[con_femalepro&(df_test.gender==0),['quantity']] = df_test.loc[con_femalepro&(df_test.gender==0),['quantity']] *1.3
reamount()

# 1,2월 할인 효과 
    # 20대의 구매 증가 및 할인된 가격
a = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['quantity']] * 1.2
df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['quantity']] = a
a_p = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['price']] * 0.95
df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['price']] = a_p
    # 나머지의 구매 증가
b = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['quantity']] * 1.05
df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['quantity']] = b
a_p = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['price']] * 0.95
df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['price']] = a_p
    # 매출 수정
reamount()

# 3월 할인 효과
    # 20대의 구매 증가
a = df_test.loc[con_2021&(df_test.month == 3)&con_age_20,['quantity']] * 1.2
df_test.loc[con_2021&(df_test.month == 3)&con_age_20,['quantity']] = a
a_p = df_test.loc[con_2021&(df_test.month == 3)&con_age_20,['quantity']] * 0.9
df_test.loc[con_2021&(df_test.month == 3)&con_age_20,['quantity']] = a_p
    # 매출 수정
reamount()

# 7, 8월 할인 효과
    # product_num이 13,14,16,20,25,28,33,35인 것들의 구매 
con_summer = ((df_test.product_num == 13)&(df_test.product_num == 14)&(df_test.product_num == 16)&(df_test.product_num == 20)&(df_test.product_num == 25)&(df_test.product_num == 28)&(df_test.product_num == 33)&(df_test.product_num == 35))
a = df_test.loc[con_summer&con_2021&((df_test.month == 7)|(df_test.month == 8)),['quantity']] * 1.5
df_test.loc[con_summer&con_2021&((df_test.month == 7)|(df_test.month == 8)),['quantity']] = a
reamount()

# 9월 할인 효과
    # 여성 의류 구매 증가 15,21,22,23,30,31,32
a = df_test.loc[con_femalepro&(df_test.gender==0)&con_2021,['quantity']] * 1.3
df_test.loc[con_femalepro&(df_test.gender==0)&con_2021,['quantity']] = a
a_p = df_test.loc[con_femalepro&(df_test.gender==0)&con_2021,['price']] * 0.8
df_test.loc[con_femalepro&(df_test.gender==0)&con_2021,['quantity']] = a_p
reamount()
df_test.quantity = df_test.quantity.astype('int')
df_test.price = df_test.price.round(2)
df_test.amount = df_test.amount.round(2)

In [18]:
df_test.head()

,cusno,gender,age,region,date,day,time,product_num,price,quantity,amount,year,month
0,685,1,18,10,2021-03-15,Monday,24,20,19598.56,26,520341.82,2021,3
1,560,1,71,10,2020-01-11,Saturday,15,33,43636.92,49,2146936.46,2020,1
2,630,1,52,3,2020-06-07,Sunday,23,10,8005.61,23,187331.27,2020,6
3,193,1,37,4,2021-12-16,Thursday,11,19,13776.14,13,192177.11,2021,12
4,836,0,35,5,2020-10-11,Sunday,18,36,45999.08,67,3091138.18,2020,10


In [25]:
df1 = df_test.copy()
df1['year'] = df1.date.dt.year
df1.set_index('date',inplace=True)
df1.sort_index(ascending=True,inplace=True)
df1.head()

,cusno,gender,age,region,day,time,product_num,price,quantity,amount,year,month
date,,,,,,,,,,,,
2020-01-01,375,1,66,1,Wednesday,21,26,25997.79,3,93592.04,2020,1
2020-01-01,632,0,74,6,Wednesday,12,24,32643.17,67,2193621.02,2020,1
2020-01-01,991,0,19,4,Wednesday,23,27,19927.21,114,2271701.94,2020,1
2020-01-01,862,1,71,10,Wednesday,13,29,22624.34,28,638006.39,2020,1
2020-01-01,196,0,27,8,Wednesday,20,24,32643.17,22,744264.28,2020,1


In [27]:
df_cus = df1.loc[:,['cusno','gender','age','region']]
df_cus = df_cus.drop_duplicates(['cusno'])
df_cus.set_index('cusno',inplace=True)
df_cus.sort_index(ascending=True,inplace=True)
df_cus.head()

,gender,age,region
cusno,,,
1,1,75,5
2,0,21,3
3,0,63,10
4,0,32,6
5,1,16,8


In [28]:
arr = np.zeros((1000,6))
columns = ['cusno','quantity_20','amount_20','quantity_21','amount_21','sales']
df_sum = pd.DataFrame(arr,columns = columns)

# cusno
new_cus = list(range(1,1001))
df_sum.cusno = new_cus

# cusno, sales를 제외한 열들
def get_sum(year,column):         # cusno마다 해당년도의 해당 열의 합을 가져오는 함수를 정의
    li = []
    for i in new_cus:
        li.append(df1.loc[(df1.cusno == i)&(df1.year == year),[column]].sum()[0])
    return li
df_sum.quantity_20 = get_sum(2020,'quantity') ; df_sum.quantity_20 = df_sum.quantity_20.astype('int')  # float길래 int로 바꿔줌
df_sum.quantity_21 = get_sum(2021,'quantity') ; df_sum.quantity_21 = df_sum.quantity_21.astype('int')
df_sum.amount_20 = get_sum(2020,'amount')
df_sum.amount_21 = get_sum(2021,'amount')

# sales
df_sum.sales = np.where(df_sum.amount_20<df_sum.amount_21,1,0)
df_sum.set_index('cusno',inplace=True)    # cusno로 인덱스 선택
df_sum.head()

,quantity_20,amount_20,quantity_21,amount_21,sales
cusno,,,,,
1,93,1282146.66,286,6457229.29,1
2,91,2179630.90,74,837996.75,0
3,413,8559157.01,296,7073777.70,0
4,244,4939641.81,220,6335653.07,1
5,229,6351096.55,19,300099.45,0


In [29]:
# 구매내역으로부터 최종 데이터프레임을 구성
# 기준은 구매건수가 아닌 고객들
df_cus_sum = pd.concat([df_cus,df_sum],axis=1)
df_cus_sum

,gender,age,region,quantity_20,amount_20,quantity_21,amount_21,sales
cusno,,,,,,,,
1,1,75,5,93,1282146.66,286,6457229.29,1
2,0,21,3,91,2179630.90,74,837996.75,0
3,0,63,10,413,8559157.01,296,7073777.70,0
4,0,32,6,244,4939641.81,220,6335653.07,1
5,1,16,8,229,6351096.55,19,300099.45,0
...,...,...,...,...,...,...,...,...
996,0,50,10,230,1816437.30,269,7705811.71,1
997,0,53,3,359,8111569.73,209,6130432.48,0
998,1,33,10,38,1361693.18,188,5405223.99,1


# 위 df_test가 최종 raw data이며, df_cus_sum은 고객별 정리 데이터입니다.

In [30]:
con_2020 = (df_test.year == 2020)
con_2021 = (df_test.year == 2021)
con_age_20 = ((df_test.age >= 20)&(df_test.age < 30))
con_age_other = ((df_test.age < 20)|(df_test.age >= 30))


In [34]:
df_test.loc[con_2020&(df_test.month == 3)&con_age_20,['amount']].describe()

,amount
count,7.800000e+01
mean,1.277812e+06
std,1.143293e+06
min,1.825680e+04
25%,4.638000e+05
50%,9.784543e+05
75%,1.708553e+06
max,5.569870e+06


In [33]:
df_test.loc[con_2021&(df_test.month == 3)&con_age_20,['amount']].describe()

,amount
count,5.900000e+01
mean,1.009926e+06
std,8.524701e+05
min,4.698965e+04
25%,4.538610e+05
50%,7.468085e+05
75%,1.343665e+06
max,4.044562e+06


In [19]:
# 원래 데이터의 20년대비 21년 quantity 변화
df_test_1 = df.copy()
df_test_1['year'] = df_test_1.date.dt.year
df_test_1['month'] = df_test_1.date.dt.month

con_2021 = (df_test_1.year == 2021)
con_2020 = (df_test_1.year == 2020)
con_age_20 = ((df_test_1.age >= 20)&(df_test_1.age < 30))
con_age_other = ((df_test_1.age < 20)|(df_test_1.age >= 30))

other_20 = df_test_1.loc[con_2020&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_other,['quantity']].mean()
other_21 = df_test_1.loc[con_2021&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_other,['quantity']].mean()
tw_20 = df_test_1.loc[con_2020&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_20,['quantity']].mean()
tw_21 = df_test_1.loc[con_2021&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_20,['quantity']].mean()
print(other_20,'\n',other_21,'\n',tw_20,'\n',tw_21)

quantity    49.652174
dtype: float64 
 quantity    50.306767
dtype: float64 
 quantity    48.246032
dtype: float64 
 quantity    49.318584
dtype: float64


In [20]:
# 바꾼 데이터의 20년대비 21년 quantity 변화
con_2021 = (df_test.year == 2021)
con_2020 = (df_test.year == 2020)
con_age_20 = ((df_test.age >= 20)&(df_test.age < 30))
con_age_other = ((df_test.age < 20)|(df_test.age >= 30))

other_20 = df_test.loc[con_2020&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['quantity']].mean()
other_21 = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_other,['quantity']].mean()
tw_20 = df_test.loc[con_2020&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['quantity']].mean()
tw_21 = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['quantity']].mean()
print(other_20,'\n',other_21,'\n',tw_20,'\n',tw_21)

quantity    43.918841
dtype: float64 
 quantity    34.8
dtype: float64 
 quantity    41.0
dtype: float64 
 quantity    37.716814
dtype: float64


In [21]:
# 원래 데이터의 20년대비 21년 amount 변화
con_2021 = (df_test_1.year == 2021)
con_2020 = (df_test_1.year == 2020)
con_age_20 = ((df_test_1.age >= 20)&(df_test_1.age < 30))
con_age_other = ((df_test_1.age < 20)|(df_test_1.age >= 30))
a = df_test_1.loc[con_2020&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_20,['amount']].mean()
b = df_test_1.loc[con_2021&((df_test_1.month == 1)|(df_test_1.month == 2))&con_age_20,['amount']].mean()
print(a,'\n',b)

amount    1349264.865
dtype: float64 
 amount    1.344261e+06
dtype: float64


In [22]:
# 바꾼 데이터의 20년대비 21년 amount 변화
con_2021 = (df_test.year == 2021)
con_2020 = (df_test.year == 2020)
con_age_20 = ((df_test.age >= 20)&(df_test.age < 30))

con_age_other = ((df_test.age < 20)|(df_test.age >= 30))
a = df_test.loc[con_2020&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['amount']].mean()
b = df_test.loc[con_2021&((df_test.month == 1)|(df_test.month == 2))&con_age_20,['amount']].mean()
print(a,'\n',b)

amount    1189906.255
dtype: float64 
 amount    1.017678e+06
dtype: float64


In [ ]:
# 현타 발생..비슷한 방법의 노가다성 데이터 바꾸기는 한번만 해도 될 것 같음.
# 다른 방식으로 데이터 바꾸는 방법이 필요
# 전년 대비 어느 정도가 변화가 의미있는지에 대한 통계적 지식 필요
# 변수 설정에 있어 조금 더 생각이 필요 